In [1]:
import pandas as pd
from src.regex_classifier import attempt_to_classify

db = pd.read_csv('../resources/acme_security_tickets.csv')

hit rate for type Cloud Resource Access: 100.00%
hit rate for type Data Export: 100.00%
hit rate for type DevTool Install: 100.00%
hit rate for type Firewall Change: 100.00%
hit rate for type Network Access: 100.00%
hit rate for type Permission Change: 100.00%
hit rate for type Vendor Approval: 100.00%


# regex performance on dataset
starting simply, how well does a simple regex recognize and classify the requests that we're getting?

In [2]:
db['regex_classification'] = db['details'].apply(attempt_to_classify).apply(lambda txt : txt.replace('_',' '))

# print(db[['details', 'request_type', 'regex_classification']])

requests_by_types = db.groupby('request_type')

for request_type, requests in requests_by_types:
    hit_rate = requests.apply(lambda row: row['request_type'].upper() == row['regex_classification'].upper(), axis=1).sum() / requests.shape[0]
    print(f"hit rate for type {request_type}: {hit_rate*100:.2f}%")

hit rate for type Cloud Resource Access: 100.00%
hit rate for type Data Export: 100.00%
hit rate for type DevTool Install: 100.00%
hit rate for type Firewall Change: 100.00%
hit rate for type Network Access: 100.00%
hit rate for type Permission Change: 100.00%
hit rate for type Vendor Approval: 100.00%


is seems like using regex just to classify the requests is **highly accurate**, but I would guess that this indicates a data issue rather than real world performance.
even so, I will try to move one and see how far I can go with just that simple stuff.

let's start by examining every mandatory field and see how we extract it from the data.

In [13]:
db[['request_type', 'mandatory_fields']].drop_duplicates().sort_values(by=['request_type'])

,request_type,mandatory_fields
8,Cloud Resource Access,Business Justification; Data Sensitivity Level
7,Data Export,Business Justification; PII Involved; Data Des...
14,DevTool Install,Business Justification; Team Lead Approval
4,Firewall Change,Business Justification; Source System; Destina...
3,Network Access,Business Justification; Source CIDR; Approval ...
0,Permission Change,Business Justification; Duration; Manager Appr...
1,Vendor Approval,Vendor Security Questionnaire; Data Classifica...


In [14]:
db[(db['request_type'] == 'Data Export')][['details', 'fields_provided', 'outcome']]

,details,fields_provided,outcome
7,Request to export anonymized user event data (...,Business Justification; Data Destination,Info Requested
10,Request to export anonymized user event data (...,Business Justification; Data Destination,Info Requested
13,Request to export anonymized user event data (...,PII Involved; Data Destination,Info Requested
39,Request to export anonymized user event data (...,PII Involved,Info Requested
43,Request to export anonymized user event data (...,Business Justification; PII Involved,Info Requested
...,...,...,...
957,Request to export anonymized user event data (...,Business Justification,Info Requested
961,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,Approved
968,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,Approved
978,Request to export anonymized user event data (...,Business Justification; PII Involved; Data Des...,Approved
